# Lesson 3.4: Practical with Vector Stores

---

In the previous lesson, we learned about the concepts of **Embeddings** and **Vector Stores**. Now, we will dive into practical exercises with some common Vector Stores in LangChain, including local solutions like **FAISS** and **Chroma**, along with an overview of cloud solutions like **Pinecone** or **Weaviate**.

## 1. Overview of Vector Stores

As learned, **Vector Stores** are specialized databases designed to store and manage vector embeddings. They play a central role in **Retrieval-Augmented Generation (RAG)** systems by enabling efficient semantic search, helping LLMs retrieve relevant information from your knowledge base.

The choice of Vector Store depends on your project's scale, performance requirements, scalability, and budget.




---

## 2. FAISS (Facebook AI Similarity Search): Local, Simple to Start Vector Store

**FAISS** is an open-source library developed by Facebook AI, providing efficient algorithms for similarity search on large sets of vectors. LangChain integrates FAISS to create a local Vector Store, which is very suitable for development and testing.

### 2.1. FAISS Characteristics

* **Local (In-memory/Disk-backed):** FAISS can operate entirely in memory or save the index to disk for later loading.
* **Simple:** Easy to set up and use for small to medium-sized cases.
* **Efficient:** Provides high search performance for large vector datasets.
* **No Distributed Scalability:** Not designed for large-scale multi-user or distributed applications.

### 2.2. Practical with FAISS

We will create a FAISS index from a set of documents and then perform a similarity search.

**Preparation:**
* Ensure you have `langchain-openai` and `faiss-cpu` installed.
* Set the `OPENAI_API_KEY` environment variable.

In [ ]:
# Install libraries if not already installed
# pip install langchain-openai openai faiss-cpu

import os
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Set environment variable for OpenAI API key
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

# 1. Initialize Embeddings model
embeddings_model = OpenAIEmbeddings(model="text-embedding-ada-002")

# 2. Prepare sample documents (similar to Lesson 3.2)
long_text_content = """
LangChain is an open-source framework designed to help developers build applications powered by Large Language Models (LLMs) more easily and efficiently. It provides a set of tools, components, and abstractions to simplify complex processes related to LLMs, from prompt management to connecting LLMs with external data sources and tools.

Key components of LangChain include:
- Models: Interfaces for LLMs and Chat Models.
- Prompts: Tools for constructing and managing prompts for LLMs.
- Chains: Connecting components together to form a processing flow.
- Agents: Allowing LLMs to make decisions and use tools.
- Retrieval: Tools for loading, splitting, embedding, and retrieving documents.
- Memory: Maintaining state and context in conversations.

Retrieval-Augmented Generation (RAG) is a popular architecture in LLM applications. RAG allows an LLM to retrieve information from an external knowledge base (often a Vector Store) before generating a response. This helps improve accuracy and reduce LLM "hallucinations."

To build a RAG system, the main steps typically include:
1. Document Loading.
2. Text Splitting into chunks.
3. Creating embeddings for text chunks.
4. Storing embeddings in a Vector Store.
5. When queried, searching for relevant chunks in the Vector Store.
6. Passing relevant chunks and the query to the LLM to generate an answer.

FAISS is an efficient similarity search library, often used as a local Vector Store in LangChain. It allows you to store embeddings and perform fast searches on them.
"""

# Create a Document from the text string
doc = Document(page_content=long_text_content, metadata={"source": "langchain_overview"})

# Split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    add_start_index=True
)
chunks = text_splitter.split_documents([doc])

print(f"Tổng số chunks sau khi chia nhỏ: {len(chunks)}\n") # Total chunks after splitting:

# 3. Create FAISS index from documents
print("Đang tạo FAISS index từ các chunks...") # Creating FAISS index from chunks...
faiss_vector_store = FAISS.from_documents(chunks, embeddings_model)
print("Đã tạo FAISS index thành công.") # FAISS index created successfully.

# 4. Perform similarity search
query_faiss = "What are the main components of LangChain?"
print(f"\nĐang tìm kiếm cho truy vấn: '{query_faiss}'") # Searching for query:
retrieved_docs_faiss = faiss_vector_store.similarity_search(query_faiss, k=2)

print("\n--- Kết quả tìm kiếm từ FAISS ---") # --- Search Results from FAISS ---
for i, doc_retrieved in enumerate(retrieved_docs_faiss):
    print(f"Tài liệu {i+1} (Nguồn: {doc_retrieved.metadata.get('source', 'Không rõ')}):") # Document {i+1} (Source: {doc_retrieved.metadata.get('source', 'Unknown')}):
    print(f"  Nội dung: {doc_retrieved.page_content[:200]}...") # Content:
    print(f"  Siêu dữ liệu: {doc_retrieved.metadata}") # Metadata:
    print("-" * 30)

# 5. Save and load FAISS index
import shutil
save_path_faiss = "faiss_index_demo"
faiss_vector_store.save_local(save_path_faiss)
print(f"\nĐã lưu FAISS index vào thư mục: {save_path_faiss}") # FAISS index saved to directory:

loaded_faiss_vector_store = FAISS.load_local(
    save_path_faiss,
    embeddings_model,
    allow_dangerous_deserialization=True # Necessary when loading from disk
)
print(f"Đã tải lại FAISS index từ thư mục: {save_path_faiss}") # FAISS index reloaded from directory:

query_loaded_faiss = "What is RAG and why is it important?"
print(f"\nĐang tìm kiếm với FAISS index đã tải lại cho truy vấn: '{query_loaded_faiss}'") # Searching with reloaded FAISS index for query:
retrieved_loaded_faiss = loaded_faiss_vector_store.similarity_search(query_loaded_faiss, k=1)
print("\n--- Kết quả tìm kiếm từ FAISS đã tải lại ---") # --- Search Results from Reloaded FAISS ---
for doc_retrieved in retrieved_loaded_faiss:
    print(f"  Nội dung: {doc_retrieved.page_content[:200]}...") # Content:
    print(f"  Siêu dữ liệu: {doc_retrieved.metadata}") # Metadata:

# Clean up index directory
if os.path.exists(save_path_faiss):
    shutil.rmtree(save_path_faiss)
    print(f"\nĐã xóa thư mục '{save_path_faiss}'.") # Directory '{save_path_faiss}' removed.


---

## 3. Chroma: Open-Source, Easy-to-Use Vector Store

**Chroma** is an open-source vector database designed to simplify the process of building LLM applications. It focuses on ease of use and provides intuitive APIs for embedding, storing, and searching vectors. It supports both local (in-memory or disk-backed) and client/server modes for larger applications.

### 3.1. Chroma Characteristics

* **Open-Source:** Completely free and customizable.
* **Easy to Use:** Simple, intuitive API.
* **Flexible Storage:** Can run entirely in memory, persist data to local disk, or be deployed as a standalone server.
* **Metadata Filtering Support:** Allows filtering search results based on document metadata. This is very useful when you want to limit searches to documents that meet certain criteria (e.g., only search in documents from a specific source or created after a certain date).

### 3.2. Practical with Chroma

We will set up ChromaDB, store documents, and perform queries.

**Preparation:**
* Ensure you have `langchain-openai` and `chromadb` installed.
* Set the `OPENAI_API_KEY` environment variable.

In [ ]:
# Install libraries if not already installed
# pip install langchain-openai openai chromadb

import os
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
import shutil

# Set environment variable for OpenAI API key
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

# 1. Initialize Embeddings model
embeddings_model = OpenAIEmbeddings(model="text-embedding-ada-002")

# 2. Prepare sample documents (similar to FAISS example)
long_text_content_chroma = """
Chroma is an open-source vector database designed to simplify the process of building LLM applications. It focuses on ease of use and provides intuitive APIs for embedding, storing, and searching vectors.

Chroma supports multiple operating modes:
- Local (in-memory) mode: Data only exists within the current session.
- Local (disk-backed) mode: Data is stored in a directory on disk, allowing persistence between sessions.
- Server (client/server) mode: Deploy Chroma as a standalone service, allowing multiple client applications to connect and access data.

An important feature of Chroma is the ability to filter search results based on metadata. This is very useful when you want to limit searches to documents that meet certain criteria (e.g., only search in documents from a specific source or created after a certain date).

Chroma also integrates well with LangChain, allowing you to easily use it as a Vector Store for your RAG applications.
"""

doc_chroma = Document(page_content=long_text_content_chroma, metadata={"source": "chroma_docs", "version": "1.0"})

# Split the document into chunks
text_splitter_chroma = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    add_start_index=True
)
chunks_chroma = text_splitter_chroma.split_documents([doc_chroma])

print(f"Tổng số chunks sau khi chia nhỏ cho Chroma: {len(chunks_chroma)}\n") # Total chunks after splitting for Chroma:

# 3. Set up ChromaDB and store documents
persist_directory = "./chroma_db_demo" # Directory to store Chroma data
# Delete old directory if it exists to ensure a clean start
if os.path.exists(persist_directory):
    shutil.rmtree(persist_directory)

print(f"Đang tạo Chroma Vector Store và lưu trữ vào: {persist_directory}...") # Creating Chroma Vector Store and storing to:
chroma_vector_store = Chroma.from_documents(
    chunks_chroma,
    embeddings_model,
    persist_directory=persist_directory
)
print("Đã tạo và lưu trữ Chroma Vector Store thành công.") # Chroma Vector Store created and stored successfully.

# 4. Query documents
query_chroma = "How does ChromaDB work and what are its modes?"
print(f"\nĐang tìm kiếm cho truy vấn: '{query_chroma}'") # Searching for query:
retrieved_docs_chroma = chroma_vector_store.similarity_search(query_chroma, k=2)

print("\n--- Kết quả tìm kiếm từ Chroma ---") # --- Search Results from Chroma ---
for i, doc_retrieved in enumerate(retrieved_docs_chroma):
    print(f"Tài liệu {i+1} (Nguồn: {doc_retrieved.metadata.get('source', 'Không rõ')}):") # Document {i+1} (Source: {doc_retrieved.metadata.get('source', 'Unknown')}):
    print(f"  Nội dung: {doc_retrieved.page_content[:200]}...") # Content:
    print(f"  Siêu dữ liệu: {doc_retrieved.metadata}") # Metadata:
    print("-" * 30)

# 5. Load Chroma Vector Store from disk and query
print(f"\nĐang tải lại Chroma Vector Store từ: {persist_directory}...") # Loading Chroma Vector Store from:
loaded_chroma_vector_store = Chroma(
    persist_directory=persist_directory,
    embedding_function=embeddings_model # Important: must pass the embedding_function again
)
print("Đã tải lại Chroma Vector Store thành công.") # Chroma Vector Store reloaded successfully.

query_loaded_chroma = "What are the benefits of Chroma's metadata filtering?"
print(f"\nĐang tìm kiếm với Chroma index đã tải lại cho truy vấn: '{query_loaded_chroma}'") # Searching with reloaded Chroma index for query:
retrieved_loaded_chroma = loaded_chroma_vector_store.similarity_search(query_loaded_chroma, k=1)
print("\n--- Kết quả tìm kiếm từ Chroma đã tải lại ---") # --- Search Results from Reloaded Chroma ---
for doc_retrieved in retrieved_loaded_chroma:
    print(f"  Nội dung: {doc_retrieved.page_content[:200]}...") # Content:
    print(f"  Siêu dữ liệu: {doc_retrieved.metadata}") # Metadata:

# Clean up Chroma directory
if os.path.exists(persist_directory):
    shutil.rmtree(persist_directory)
    print(f"\nĐã xóa thư mục Chroma '{persist_directory}'.") # Chroma directory '{persist_directory}' removed.


---

## 4. Pinecone/Weaviate (Optional): Introduction to Cloud Vector Stores

For large-scale, production LLM applications, you will need Vector Stores that offer scalability, high durability, and easy management. **Pinecone** and **Weaviate** are two of the leading cloud-based Vector Stores.

### 4.1. Architecture and Benefits Overview

* **Architecture:** Both Pinecone and Weaviate are managed services running in the cloud. They provide APIs for you to interact with, eliminating the need for you to manage infrastructure yourself.
* **Benefits:**
    * **Scalability:** Easily scale to handle millions or billions of vectors without worrying about resources.
    * **Durability:** Data is backed up and protected, ensuring no loss.
    * **High Performance:** Optimized for high-speed similarity queries.
    * **Reliability:** Managed by the provider, ensuring high uptime.
    * **Advanced Features:** Support complex filtering, real-time data updates, and index management features.



### 4.2. Basic Integration (Conceptual, Not for Deep Implementation)

Integrating Pinecone or Weaviate with LangChain typically involves the following steps:

1.  **Sign up for an account and get API Key:** You need to register an account on their platform (Pinecone, Weaviate Cloud) and obtain an API key along with other configuration information (e.g., environment, endpoint URL).
2.  **Install Python library:** Install the corresponding Python SDK package (e.g., `pinecone-client`, `weaviate-client`).
3.  **Set environment variables:** Store API keys and other configuration information in environment variables.
4.  **Initialize Vector Store in LangChain:** Use LangChain's integration class (e.g., `PineconeVectorStore`, `WeaviateVectorStore`) and pass the necessary configuration information.
5.  **Add and query documents:** Use methods similar to FAISS or Chroma (`from_documents`, `similarity_search`, `as_retriever`).

**Configuration Example (will not run without actual account and API keys):**

In [ ]:
# Pinecone configuration example (will not run without actual account and API keys)
# Install if not already installed: pip install pinecone-client langchain-pinecone

# import os
# from langchain_openai import OpenAIEmbeddings
# from langchain_pinecone import PineconeVectorStore
# from langchain_core.documents import Document

# os.environ["PINECONE_API_KEY"] = "YOUR_PINECONE_API_KEY"
# os.environ["PINECONE_ENVIRONMENT"] = "YOUR_PINECONE_ENVIRONMENT" # e.g.: "gcp-starter"
# index_name = "my-langchain-index"

# embeddings_model = OpenAIEmbeddings(model="text-embedding-ada-002")

# # Initialize Pinecone
# # from pinecone import Pinecone, ServerlessSpec
# # pinecone = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

# # if index_name not in pinecone.list_indexes():
# #     pinecone.create_index(
# #         name=index_name,
# #         dimension=embeddings_model.client.dimensions, # Vector size of the embedding model
# #         metric='cosine',
# #         spec=ServerlessSpec(cloud='aws', region='us-east-1') # Or StarterSpec()
# #     )

# # Connect to PineconeVectorStore
# # vector_store_pinecone = PineconeVectorStore.from_existing_index(
# #     index_name=index_name,
# #     embedding=embeddings_model
# # )

# # Or if you want to add new documents:
# # documents_pinecone = [Document(page_content="Pinecone is a cloud vector database.", metadata={"source": "pinecone_docs"})]
# # vector_store_pinecone = PineconeVectorStore.from_documents(
# #     documents_pinecone,
# #     embeddings_model,
# #     index_name=index_name
# # )

# # query_pinecone = "Which cloud vector database is popular?"
# # retrieved_docs_pinecone = vector_store_pinecone.similarity_search(query_pinecone, k=1)
# # print(f"Results from Pinecone: {retrieved_docs_pinecone}")

The choice of the appropriate Vector Store is a crucial decision in designing your RAG application. For small projects and learning, local FAISS and Chroma are excellent choices. For production applications requiring scalability and durability, cloud solutions like Pinecone or Weaviate are necessary.


---

## Lesson Summary

This lesson provided a practical insight into working with **Vector Stores** in LangChain. We practiced in detail with **FAISS**, a simple yet efficient local Vector Store, including creating an index, performing similarity searches, and saving/loading the index. Next, we explored **Chroma**, a flexible open-source Vector Store, supporting both local and client/server storage, and performed similar operations. Finally, the lesson provided an overview of large-scale cloud Vector Stores like **Pinecone** and **Weaviate**, highlighting their architecture, benefits, and basic integration. Mastering these Vector Stores is the final and crucial step for you to build a complete and effective RAG system.